In [2]:
%load_ext autoreload
%autoreload

import os, sys

modules = os.path.abspath(os.path.join("..", "src"))
sys.path.append(modules)

import nlp_utils, keywords_utils, load_utils, run_analysis
from config import *

/home/msalvetti/notebooks_2/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2026-02-04 16:59:22.141492: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
import polars as pl
patents = pl.read_parquet("../data/processed/patents_with_ID.parquet")

ids =patents.filter(
    pl.col("ID").str.starts_with("CHCHE")
    ).filter(pl.col("Abstract").is_not_null()).group_by("ID").agg(
        pl.col("Abstract").str.join("\n").alias("text")).select("ID").unique()["ID"]
#.to_pandas().to_parquet("abstract_LLMs.parquet")

In [4]:
run = pl.scan_parquet("../data/cleaned/grouped_text/*.parquet")

In [5]:
run.filter(pl.col("ID").is_in(ids)).filter(pl.col("text")!="").sink_parquet("companies_patent_LLM.parquet")

/tmp/ipykernel_200443/379810147.py:1: DeprecationWarning: `is_in` with a collection of the same datatype is ambiguous and deprecated.
Please use `implode` to return to previous behavior.

See https://github.com/pola-rs/polars/issues/22149 for more information.
  run.filter(pl.col("ID").is_in(ids)).filter(pl.col("text")!="").sink_parquet("companies_patent_LLM.parquet")
